In [6]:
(require 'macro)

ok

#### Writer Note:
The Calysto Scheme kernel does not use the `(define-macro)` syntax. Instead, Calysto Scheme kernel uses `(define-syntax)` to create macros.

# Macros

Macros are a feature of Scheme that allows us to define new special forms in the language. In Python, we have a certain inventory of expressions and statement types. The same is true with Scheme, we had defined `lambda`, `if`, `con`, `and`, `or`, etc. Everything else is a procedure. 

Macros allow us to extend the inventory of expressions and statement types by inventing new kinds of special forms. This means we can change the way the language works, inventing new ways in which the flow of evaluation proceeds through a program.

Macros work well in Lisp languages such as Scheme since it's easy to view code as data. Programs are just lists. 

## Macros Perform Code Transformations

The way we'll invent new special forms is to describe how to take parts of the special forms and construct a regular piece of Scheme code out of them and evaluate it.

A macro is an operation performed on the source code of a program before evaluation. Macros exist in many languages, but are easiest to define correctly in a language like Lisp where the code is just data.

Scheme has a special form `define-macro` that defines a source code transformation. An example is as the following,

In [1]:
(define-syntax define-macro
  (syntax-rules ()
    ((define-macro (name . args) body ...)
     (define-syntax name
       (rsc-macro-transformer
         (let ((transformer (lambda args body ...)))
           (lambda (exp env)
              (apply transformer (cdr exp)))))))))

In [2]:
; This is the define-macro syntax. It's included here for educational purpose and assignments

(define-macro (twice expr)
 (list 'begin expr expr))


Traceback (most recent call last):
  File "In [2]", line 3, col 1
ParseError: no matching clause found for (define-macro (twice expr) (list (quote begin) expr expr))



In [15]:
; This is the define-syntax version that works for the Calysto Scheme. Use this version when actually executing the cells
; in the notebook

(define-syntax twice
  [(twice ?expr) (list 'begin ?expr ?expr)])

As we can see above, it looks very similar to normal procedure definition. The procedure says that when we `twice` an expression `expr`, the program builds a list that starts with `'begin` followed by the `expr` twice. 

Once we've defined the macro, we can `twice` anything. For example,

In [16]:
(twice (print 2))

2
2


(begin <void> <void>)

Since `twice` is a macro, it evaluates the body of the macro,

In [ ]:
(list 'begin expr expr)

On the expression that we put in as an input,

In [ ]:
(print 2)

...before the expression itself is evaluated. Thus, the output would be,

In [ ]:
(begin (print 2) (print 2))

The program then evaluates the expression above, which means the program will `print 2` twice.

In [ ]:
> (twice (print 2))
2
2

If `twice` were a normal procedure instead of a macro, the expression `(print 2)` within `(twice (print 2))` would have been evaluated before `twice` were called. That way, `2` would only be printed once.

The evaluation procedure of a macro call expression is as the following:
1. Evaluate the operator sub-expression, which evaluates to a macro
2. Call the macro procedure on the operand expressions **without evaluating them first**
3. Evaluate the expression returned from the macro procedure. 

Thus, macros takes in an expression and returns an expression instead of taking in values and returning values.

## Demo

In [1]:
(print 2)

2


Above, when we `print 2`, Scheme simply prints 2.

In [2]:
(define x (print 2))

2


In [3]:
x

Above, when we define `x`, Scheme prints 2. However, the `print` itself returns nothing. Thus, `x` is bound to nothing.

If we want to `print 2` twice, we can use the `begin` keyword,

In [4]:
(begin (print 2) (print 2))

2
2


Let's say we're stuck with only procedures and that we want to define `twice`. 

In [19]:
(define (twice expr)
  (list 'begin expr expr))

If we use the `twice` definition above,

In [20]:
(twice (print 2))

2
2


(begin <void> <void>)

Then Scheme prints `2` and constructs the expression `(begin None None)`. If we want `twice` to return the output that we desired, we would have to quote `'` the argument `expr`,

In [8]:
(twice '(print 2))

(begin (print 2) (print 2))

If we want to evaluate the result of `print 2` above, it can be done by using `eval`,

In [9]:
(eval (twice '(print 2)))

2
2


The idea of macro is that the quotation `'` and evaluation procedure is already taken into account. 

In [21]:
(define-macro (twice expr)
  (list 'begin expr expr))


Traceback (most recent call last):
  File "In [21]", line 1, col 22
RunTimeError: unbound variable 'expr'



With the `twice` implementation above, if we call `twice` on `print 2` once more,

In [ ]:
> (twice (print 2))
2
2

The program makes sure that `print 2` is not evaluated (e.g. by quoting it). We've changed the way the language works. `twice` is now a special form that doesn't involve evaluating the arguments first. This means we have complete control over when and how codes are evaluated inside a macro. 

What's an application of a macro?

Let's say we want to define a procedure that checks whether something is `True` or `False`.

In [12]:
(define (check val)
  (if val
      'passed
      'failed
      ))

Now if we have `x` of `-2` and we want to check if `x` is greater than 0, we would have failed the check.

In [13]:
(define x -2)
(check (> x 0))

failed

The program works! Unfortunately, it doesn't tell us which part fails when the code fails. In order to do that, we need access to the expression that was evaluated, `(> x 0)`. In order to do that, we would need a macro. 

In [ ]:
; The define-macro version
(define-macro (check expr)
 (list 'if expr ''passed ''fail)

In [22]:
; The define-syntax version
(define-syntax check
  [(check ?expr) (list 'if ?expr ''passed ''fail)])

In [23]:
(define x -2)
(check (> x 0))

(if #f (quote passed) (quote fail))